In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By

driver = webdriver.Chrome()

driver.get("https://www.imdb.com/chart/top/")

title = driver.title

driver.implicitly_wait(0.5)


container = driver.find_element(by=By.XPATH, value="//*[@id='__next']/main/div/div[3]/section/div/div[2]/div/ul")

movies = container.find_elements(by=By.CLASS_NAME, value='ipc-metadata-list-summary-item.sc-4929eaf6-0.DLYcv.cli-parent')


data_dicts = []
for movie in movies:
    # Obtener título
    title = movie.find_element(by=By.CLASS_NAME, value='ipc-title-link-wrapper')
    clean_title = title.text.split(' ', 1)[1].upper()

    # Obtener año
    year = movie.find_element(by=By.XPATH, value='.//div[@class="sc-300a8231-6 dBUjvq cli-title-metadata"]//span[1]')


    # Obtener duración
    duration = movie.find_element(by=By.XPATH, value='.//div[@class="sc-300a8231-6 dBUjvq cli-title-metadata"]//span[2]')

    # Obtener rating
    rating = movie.find_element(by=By.CLASS_NAME, value='ipc-rating-star--rating')

    # Agrupar los datos en un solo diccionario
    movie_data = {
        'TITLE': clean_title,
        'YEAR': year.text,
        'DURATION': duration.text,
        'RATING': rating.text
    }

    # Agregar el diccionario a la lista
    data_dicts.append(movie_data)
driver.quit()

# Imprimir el resultado

In [ ]:
import pandas as pd


df = pd.DataFrame(data_dicts)
df.to_excel('top_250_movies.xlsx', index=False)


